In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np
import scipy.sparse
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from numpy import hstack
from numpy import vstack
from numpy import asarray
from pandas.core.common import flatten

from sklearn.model_selection import KFold



from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve,accuracy_score,confusion_matrix,recall_score,precision_score,f1_score, auc, roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV

# Our algorithms, by from the easiest to the hardest to intepret.
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost.sklearn import XGBClassifier
from mlens.ensemble import SuperLearner
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier, cv, Pool

import os


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df =pd.read_csv('/kaggle/input/malware-executable-detection/uci_malware_detection.csv')
print(df.shape)
df.head()

In [ ]:
df['Label'] = df['Label'].map({'non-malicious': 1, 'malicious': 0})
print(df.shape)
df.head()

In [ ]:
df = df.drop_duplicates(keep=False)
print(df.shape)

In [ ]:
# Get X, y
y = df["Label"]
X = df.drop("Label", axis=1)

In [ ]:
print("Target Attribute distribution \n")
print(df.Label.value_counts(),"\n")

fig,ax= plt.subplots()
fig.set_size_inches(20,5)
sns.countplot(x= "Label",data=df,ax= ax)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
# 1. Logistic Regression
lr_model =  LogisticRegression(max_iter=500 ,random_state=42)

# 2. Decision Tree
dt_model =  DecisionTreeClassifier()

# 3. Naive Bayes
nb_model = GaussianNB()

# 4. Random Forest
rf_model = RandomForestClassifier( n_jobs=-1)

# 5. KNN
knn_model = KNeighborsClassifier(n_jobs=-1)

# 6. AdaBoost
ada_boost_model = AdaBoostClassifier()

# 7. XgBoost
xg_boost_model = XGBClassifier()

# 8. Bagging
bagging_model = BaggingClassifier()

# 9. ExtraTrees 
extra_trees_model = ExtraTreesClassifier()

# 10. SVM
#svm_model = SVC(probability=True)

# 11. LGBM classifier
lgbm_model = LGBMClassifier(random_state = 16)

# 12. Categorical Boosting
catboost_model = CatBoostClassifier()

######################### LIST OF ALL MODELS #############################3
ensemble_clf=[lr_model, dt_model, nb_model, rf_model, knn_model, ada_boost_model, \
              xg_boost_model, bagging_model,extra_trees_model,lgbm_model ] 

#print(ensemble_clf)
print(len(ensemble_clf))

In [ ]:
for i in range(len(ensemble_clf)):
    # Initialising model
    print('#############################'+str(type(ensemble_clf[i]).__name__)+'#############################')
    #Fitting on data
    ensemble_clf[i].fit(X_train, y_train)

    #Scoring the model on train data
    print("Training Accuracy :\t ", ensemble_clf[i].score(X_train, y_train))

    #Scoring the model on test_data
    print("Testing Accuracy :\t  ",  ensemble_clf[i].score(X_test, y_test))

    y_pred = ensemble_clf[i].predict(X_test)
    print(accuracy_score(y_test, y_pred))
    print('ROC-AUC score', roc_auc_score(y_test,y_pred))
    print('Confusion matrix:\n ', confusion_matrix(y_test, y_pred))

### Super Classifier

In [ ]:
# create a list of base-models

def get_models():
    models = list()
    models.append(LogisticRegression(max_iter=500 ,random_state=42))
    models.append(dt_model)
    #models.append(SVC(gamma='scale', probability=True))
    models.append(nb_model)
    #models.append(knn_model)
    models.append(ada_boost_model)
    models.append(xg_boost_model)
    models.append(rf_model)
    models.append(bagging_model)
    models.append(extra_trees_model)
    return models


# create the super learner
def get_super_learner(X):
    ensemble = SuperLearner(scorer=accuracy_score, folds=10, sample_size=len(X))
    # add base models
    models = get_models()
    ensemble.add(models)
    # add the meta model
    #ensemble.add_meta(LogisticRegression(solver='lbfgs'))
    ensemble.add_meta(LogisticRegression(solver='lbfgs'))
    return ensemble


print('Train', X_train.shape, y_train.shape, 'Test', X_test.shape, y_test.shape)
# create the super learner
ensemble_mlens = get_super_learner(X_train)
# fit the super learner
ensemble_mlens.fit(X_train, y_train)
# summarize base learners
print(ensemble_mlens.data)
# make predictions on hold out set
yhat = ensemble_mlens.predict(X_test)
print('Super Learner: %.3f' % (accuracy_score(y_test, yhat) * 100))
print('ROC-AUC score', roc_auc_score(y_test, yhat))
print('Confusion matrix:\n ', confusion_matrix(y_test, yhat))